In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten

Using TensorFlow backend.


In [5]:
model = Sequential()
#convolutional
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#fully connected layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

#output layer
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 18496)             0         
__________

In [6]:
batch_size = 16

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory('male_female/train', target_size=(150, 150),
                                                    batch_size=batch_size, class_mode='binary')
validation_generator = test_datagen.flow_from_directory('male_female/validation', target_size=(150, 150),
                                                        batch_size=batch_size, class_mode='binary')

Found 1960 images belonging to 2 classes.
Found 840 images belonging to 2 classes.


In [9]:
callback = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, 
                                       write_grads=False, write_images=False, embeddings_freq=0, 
                                       embeddings_layer_names=None, embeddings_metadata=None)

In [10]:
model.fit_generator(train_generator, steps_per_epoch=1960//batch_size, epochs=30,
                    validation_data=validation_generator, validation_steps=840//batch_size, callbacks=[callback])

Epoch 1/30
122/122 [==============================] - 146s 1s/step - loss: 0.4362 - acc: 0.7951 - val_loss: 0.2408 - val_acc: 0.9038
Epoch 2/30
122/122 [==============================] - 134s 1s/step - loss: 0.2524 - acc: 0.9109 - val_loss: 0.2319 - val_acc: 0.9279
Epoch 3/30
122/122 [==============================] - 136s 1s/step - loss: 0.2305 - acc: 0.9114 - val_loss: 0.2157 - val_acc: 0.9243
Epoch 4/30
122/122 [==============================] - 181s 1s/step - loss: 0.1823 - acc: 0.9283 - val_loss: 0.2446 - val_acc: 0.8930
Epoch 5/30
122/122 [==============================] - 169s 1s/step - loss: 0.1833 - acc: 0.9273 - val_loss: 0.2142 - val_acc: 0.9147
Epoch 6/30
122/122 [==============================] - 141s 1s/step - loss: 0.1509 - acc: 0.9421 - val_loss: 0.2476 - val_acc: 0.8942
Epoch 7/30
122/122 [==============================] - 139s 1s/step - loss: 0.1305 - acc: 0.9554 - val_loss: 0.2508 - val_acc: 0.9231
Epoch 8/30
122/122 [==============================] - 182s 1s/step - 

# Benchmark

1, conv 64 'relu', layer 64 'relu', layer 1 'sigmoid' 
   epocha 10, batch size 16, optimizer 'rmsprop'
   loss: 0.1637 - acc: 0.9467 - val_loss: 0.2429 - val_acc: 0.9135

2, conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 10, batch size 16, optimizer 'rmsprop'
   loss: 0.1403 - acc: 0.9595 - val_loss: 0.2117 - val_acc: 0.9026
   
3, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 10, batch size 16, optimizer 'rmsprop'
   loss: 0.1287 - acc: 0.9513 - val_loss: 0.2827 - val_acc: 0.9038
   
4, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 20, batch size 16, optimizer 'rmsprop'
   loss: 0.0752 - acc: 0.9744 - val_loss: 0.2794 - val_acc: 0.9363
   
5, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 30, batch size 16, optimizer 'rmsprop'
   loss: 0.0750 - acc: 0.9785 - val_loss: 0.2379 - val_acc: 0.9303
   
6, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 50, batch size 16, optimizer 'rmsprop'
   loss: 0.0984 - acc: 0.9810 - val_loss: 0.8020 - val_acc: 0.9327
   

In [32]:
model.save_weights('male_female_recognition_model.h5')